### MANUAL ANNOTATIONS

In [ ]:
import cv2
import os
import numpy as np

In [ ]:
class AnnotationTool:
    def __init__(self):
        # Parameters for the YOLO format
        self.class_names = ['NIK','Nama','Alamat']
        
        # Dataset structure
        self.base_dir = 'datasetKTP'
        self.train_images = os.path.join(self.base_dir, 'train', 'images')
        self.train_labels = os.path.join(self.base_dir, 'train', 'labels')
        self.val_images = os.path.join(self.base_dir, 'val', 'images')
        self.val_labels = os.path.join(self.base_dir, 'val', 'labels')
        
        # Create necessary directories
        os.makedirs(self.train_images, exist_ok=True)
        os.makedirs(self.train_labels, exist_ok=True)
        os.makedirs(self.val_images, exist_ok=True)
        os.makedirs(self.val_labels, exist_ok=True)
        
        # Create classes.txt
        self.create_classes_file()
        
        # Drawing variables
        self.drawing = False
        self.ix, self.iy = -1, -1
        self.boxes = []
        self.current_class = 0
        self.img = None
        self.current_image = None
        self.current_mode = 'val'  # Default mode # ganti ini untuk excute train
        
    def create_classes_file(self):
        """Create classes.txt file in dataset directory"""
        classes_file = os.path.join(self.base_dir, 'classes.txt') #nama class KTP untuk NIK Nama dan Alamat
        with open(classes_file, 'w') as f:
            f.write('\n'.join(self.class_names))
            
    def draw_all_boxes(self, img):
        """Draw all existing boxes with their labels"""
        img_copy = img.copy()
        for idx, (x1, y1, x2, y2, class_id) in enumerate(self.boxes):
            color = (0, 255, 0)  # Green color for boxes
            cv2.rectangle(img_copy, (x1, y1), (x2, y2), color, 2)
            # Add label with class name
            label = f"{self.class_names[class_id]} ({idx})"
            cv2.putText(img_copy, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        return img_copy

    def mouse_callback(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.drawing = True
            self.ix, self.iy = x, y
            
        elif event == cv2.EVENT_MOUSEMOVE:
            if self.drawing:
                img_copy = self.draw_all_boxes(self.img)
                cv2.rectangle(img_copy, (self.ix, self.iy), (x, y), (0, 255, 0), 2)
                cv2.imshow("Image", img_copy)
                
        elif event == cv2.EVENT_LBUTTONUP:
            self.drawing = False
            x1, y1 = min(self.ix, x), min(self.iy, y)
            x2, y2 = max(self.ix, x), max(self.iy, y)
            self.boxes.append((x1, y1, x2, y2, self.current_class))
            img_copy = self.draw_all_boxes(self.img)
            cv2.imshow("Image", img_copy)

    def load_existing_annotation(self, image_file, mode):
        """Load existing annotation if available"""
        h, w = self.img.shape[:2]
        labels_dir = self.train_labels if mode == 'train' else self.val_labels
        annotation_file = os.path.join(labels_dir, 
                                     image_file.replace('.jpg', '.txt').replace('.png', '.txt'))
        self.boxes = []
        if os.path.exists(annotation_file):
            with open(annotation_file, 'r') as f:
                for line in f:
                    class_id, x_center, y_center, width, height = map(float, line.strip().split())
                    # Convert from YOLO format to pixel coordinates
                    x1 = int((x_center - width/2) * w)
                    y1 = int((y_center - height/2) * h)
                    x2 = int((x_center + width/2) * w)
                    y2 = int((y_center + height/2) * h)
                    self.boxes.append((x1, y1, x2, y2, int(class_id)))

    def save_annotation(self, image_file, mode):
        """Save annotations in YOLO format"""
        h, w = self.img.shape[:2]
        annotation = []
        for box in self.boxes:
            x1, y1, x2, y2, class_id = box
            # Convert to YOLO format (normalized values)
            x_center = (x1 + x2) / (2.0 * w)
            y_center = (y1 + y2) / (2.0 * h)
            width = abs(x2 - x1) / w
            height = abs(y2 - y1) / h
            annotation.append(f"{class_id} {x_center} {y_center} {width} {height}")
        
        labels_dir = self.train_labels if mode == 'train' else self.val_labels
        annotation_file = os.path.join(labels_dir, 
                                     image_file.replace('.jpg', '.txt').replace('.png', '.txt'))
        with open(annotation_file, 'w') as f:
            f.write('\n'.join(annotation))

    def show_help(self):
        """Show help message with keyboard shortcuts"""
        help_text = """
        Keyboard shortcuts:
        'n' - Next image
        'b' - Previous image
        'c' - Change class
        'd' - Delete last box
        'r' - Reset all boxes
        's' - Save annotations
        'm' - Switch mode (train/val)
        'q' - Quit
        'h' - Show this help
        
        Current mode: {}
        """.format(self.current_mode)
        print(help_text)

    def run(self):
        """Main function to run the annotation tool"""
        self.show_help()
        
        while True:
            # Get current working directory based on mode
            current_dir = self.train_images if self.current_mode == 'train' else self.val_images
            image_files = [f for f in os.listdir(current_dir) 
                         if f.endswith(('.jpg', '.jpeg', '.png'))]
            
            if not image_files:
                print(f"No images found in {current_dir}")
                if self.current_mode == 'train':
                    self.current_mode = 'val'
                    continue
                else:
                    break
                    
            current_idx = 0
            while current_idx < len(image_files):
                image_file = image_files[current_idx]
                img_path = os.path.join(current_dir, image_file)
                self.img = cv2.imread(img_path)
                self.current_image = image_file
                
                # Load existing annotations if available
                self.load_existing_annotation(image_file, self.current_mode)
                
                # Show image with existing annotations
                img_copy = self.draw_all_boxes(self.img)
                cv2.imshow("Image", img_copy)
                cv2.setMouseCallback("Image", self.mouse_callback)

                print(f"\nAnnotating: {image_file} ({self.current_mode} set)")
                print(f"Current class: {self.class_names[self.current_class]}")

                while True:
                    key = cv2.waitKey(1) & 0xFF

                    if key == ord('n'):  # Next image
                        self.save_annotation(image_file, self.current_mode)
                        current_idx += 1
                        break
                    elif key == ord('b'):  # Previous image
                        self.save_annotation(image_file, self.current_mode)
                        current_idx = max(0, current_idx - 1)
                        break
                    elif key == ord('c'):  # Change class
                        self.current_class = (self.current_class + 1) % len(self.class_names)
                        print(f"Current class: {self.class_names[self.current_class]}")
                    elif key == ord('d'):  # Delete last box
                        if self.boxes:
                            self.boxes.pop()
                            img_copy = self.draw_all_boxes(self.img)
                            cv2.imshow("Image", img_copy)
                    elif key == ord('r'):  # Reset all boxes
                        self.boxes = []
                        img_copy = self.draw_all_boxes(self.img)
                        cv2.imshow("Image", img_copy)
                    elif key == ord('s'):  # Save current
                        self.save_annotation(image_file, self.current_mode)
                        print("Annotations saved!")
                    elif key == ord('m'):  # Switch mode
                        self.save_annotation(image_file, self.current_mode)
                        self.current_mode = 'val' if self.current_mode == 'train' else 'train'
                        print(f"Switched to {self.current_mode} mode")
                        break
                    elif key == ord('h'):  # Show help
                        self.show_help()
                    elif key == ord('q'):  # Quit
                        cv2.destroyAllWindows()
                        return

                cv2.destroyAllWindows()

if __name__ == "__main__":
    tool = AnnotationTool()
    tool.run()


        Keyboard shortcuts:
        'n' - Next image
        'b' - Previous image
        'c' - Change class
        'd' - Delete last box
        'r' - Reset all boxes
        's' - Save annotations
        'm' - Switch mode (train/val)
        'q' - Quit
        'h' - Show this help
        
        Current mode: val
        

Annotating: 3276055209990005.jpg (val set)
Current class: NIK
Current class: Nama
Current class: Alamat
Current class: NIK
Current class: Nama
Current class: Alamat
Current class: NIK
Annotations saved!

Annotating: 3278023003810008.jpg (val set)
Current class: NIK
Current class: Nama
Current class: Alamat
Current class: NIK
Annotations saved!

Annotating: 3301084206980004.jpg (val set)
Current class: NIK
Current class: Nama
Current class: Alamat
Current class: NIK
Current class: Nama
Current class: Alamat
Current class: NIK
Annotations saved!

Annotating: 3301224610000006.jpg (val set)
Current class: NIK
Current class: Nama
Current class: Alamat
Current class